In [14]:
import numpy as np
from itertools import combinations

In [15]:
def gf2_row_reduce(A: np.ndarray):
    """
    Row-reduce A over GF(2) (mod 2) and return (R, pivots),
    where pivots is a list of pivot column indices.
    """
    A = (A % 2).astype(np.uint8).copy()
    m, n = A.shape
    r = 0
    pivots = []

    for c in range(n):
        # find pivot row with a 1 in column c at/below row r
        pivot = None
        for i in range(r, m):
            if A[i, c] == 1:
                pivot = i
                break
        if pivot is None:
            continue

        # swap pivot row up
        if pivot != r:
            A[[r, pivot]] = A[[pivot, r]]

        # eliminate all other 1s in this column
        for i in range(m):
            if i != r and A[i, c] == 1:
                A[i, :] ^= A[r, :]  # XOR row op

        pivots.append(c)
        r += 1
        if r == m:
            break

    return A, pivots

In [16]:
def gf2_inv(M: np.ndarray):
    """
    Invert a square matrix over GF(2). Raises ValueError if singular.
    """
    M = (M % 2).astype(np.uint8)
    n = M.shape[0]
    if M.shape != (n, n):
        raise ValueError("gf2_inv expects a square matrix.")

    # Augment with identity: [M | I]
    A = np.concatenate([M.copy(), np.eye(n, dtype=np.uint8)], axis=1)

    r = 0
    for c in range(n):
        # find pivot
        pivot = None
        for i in range(r, n):
            if A[i, c] == 1:
                pivot = i
                break
        if pivot is None:
            raise ValueError("Matrix is singular over GF(2).")

        if pivot != r:
            A[[r, pivot]] = A[[pivot, r]]

        # eliminate
        for i in range(n):
            if i != r and A[i, c] == 1:
                A[i, :] ^= A[r, :]
        r += 1

    inv = A[:, n:] % 2
    return inv.astype(np.uint8)

In [17]:
def is_invertible_gf2(M: np.ndarray):
    """
    Check invertibility over GF(2) by row reduction (rank == n).
    """
    M = (M % 2).astype(np.uint8)
    n = M.shape[0]
    if M.shape != (n, n):
        return False
    _, pivots = gf2_row_reduce(M)
    return len(pivots) == n


In [18]:
def systematic_from_H(H: np.ndarray):

    H = (H % 2).astype(np.uint8)
    m, n = H.shape

    # check full row-rank (as assumed in the exercise)
    _, pivots = gf2_row_reduce(H)
    rank = len(pivots)
    if rank != m:
        raise ValueError(f"H is not full row-rank over GF(2): rank={rank}, m={m}")

    K = n - m
    cols = list(range(n))

    # Pick m columns to become the left block B (parity part), so we can make [I | P]
    for B_cols in combinations(cols, m):
        B_cols = list(B_cols)
        A_cols = [c for c in cols if c not in B_cols]

        # Lecture order: [B | A] = [parity | info]
        perm = B_cols + A_cols
        H_perm = H[:, perm]

        B = H_perm[:, :m]   # m×m
        A = H_perm[:, m:]   # m×K

        # Need B invertible over GF(2)
        try:
            B_inv = gf2_inv(B)
        except ValueError:
            continue

        # P = B^{-1} A so that (B^{-1} H_perm) = [I | P]
        P = (B_inv @ A) % 2

        # Lecture-form parity-check and generator matrices
        H_hat = np.hstack([np.eye(m, dtype=np.uint8), P])      # [I_m | P]
        G = np.vstack([P, np.eye(K, dtype=np.uint8)])          # [P; I_K]

        # sanity check: must satisfy H_hat @ G = 0
        if np.any((H_hat @ G) % 2):
            continue

        return H_hat.astype(np.uint8), G.astype(np.uint8), perm

    raise ValueError("Could not find a column permutation making an invertible parity block.")


In [22]:
if __name__ == "__main__":
    H = np.array([
        [1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0]
    ], dtype=np.uint8)

    H_hat, G, perm = systematic_from_H(H)

    print("Original H:\n", H.astype(int))
    print("\nColumn permutation (new order of columns):", perm)
    print("\nH_hat:\n", H_hat.astype(int))
    print("\nG:\n", G.astype(int))


Original H:
 [[1 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 0 0 1 1 0]]

Column permutation (new order of columns): [0, 1, 2, 3, 4, 5]

H_hat:
 [[1 0 0 1 1 0]
 [0 1 0 1 1 1]
 [0 0 1 1 0 1]]

G:
 [[1 1 0]
 [1 1 1]
 [1 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]
